In [1]:
%pip install langchain langchain-openai python-dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached tiktoken-0.9.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
Using cached packaging-24.2-py3-none-any.whl (65 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 kB 46.6 MB/s eta 0:00:00
Using cached tiktoken-0.9.0-cp310-cp310-macosx_11_0_arm64.whl (1.0 MB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)
  Attempting uninstall: packaging━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/16 [SQLAlchemy]
    F

In [8]:
from dotenv import load_dotenv
load_dotenv() #.env 파일 내용을 환경변수로 로드
import os
os.getenv('OPENAI_API_KEY')[:4]

'sk-p'

```
Langchain의 핵심컴포넌트 : 모델 호출단계를 구성하는 추상화 요소를 제공
    - PromptTemplate : LLM에 보낼 입력 프롬프트
    - ChatOpenAI : openai의 GPT- 모델 호출
    - Runnable : 실행가능한 객체에 대한 공통 인터페이스 -> invoke() 메소드를 통해서 입력 -> 출력 지원
    - StrOutPutParser : 문자열 출력 파서
파이프로 연결 가능 ... ex) prompt |(파이프) llm | strparser
```

In [10]:
from langchain_core.prompts import PromptTemplate
template = "{product} 제품을 생산하는 회사 이름을 뭘로 하면 좋을까?"
prompt = PromptTemplate.from_template(template)
formated_prompt = prompt.format(product = '커피')
formated_prompt

'커피 제품을 생산하는 회사 이름을 뭘로 하면 좋을까?'

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o-mini',temperature=0) # Runnable 객체 --> invoke()
response = llm.invoke([('human','안녕')]) #리스트 형식으로 해서 튜플로 입력
response


AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BbG6HbFgSRoQUl3k48rz1vNMvA3Xp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--00fb0c78-aed1-47fc-a957-3ec31aef1c09-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
# 문자열 출력 파서 Runnable 객체 - invoke
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parsed_text = parser.invoke(response)
parsed_text

'안녕하세요! 어떻게 도와드릴까요?'

LangChain Expression Language(LCEL) 단일 체인 실행

In [13]:
chain = prompt | llm | parser # runnable 객체 --> invoke
result = chain.invoke({"product":"coffee"})
print(result)

커피 제품을 생산하는 회사 이름으로는 다음과 같은 아이디어를 고려해볼 수 있습니다:

1. **커피의 정원** (Garden of Coffee)
2. **아침의 향기** (Morning Aroma)
3. **커피 이야기** (Coffee Tales)
4. **한 잔의 여유** (A Cup of Leisure)
5. **커피 마법** (Coffee Magic)
6. **원두의 미소** (Bean's Smile)
7. **커피의 예술** (Art of Coffee)
8. **커피 여행** (Coffee Journey)
9. **향긋한 순간** (Fragrant Moments)
10. **커피의 꿈** (Dream of Coffee)

이름을 정할 때는 브랜드의 이미지와 목표, 타겟 고객층을 고려하는 것이 중요합니다. 어떤 느낌을 주고 싶은지 생각해보세요!


```
LangChain을 활용한 모델 사용, 비용 모니터링 및 캐싱 전략
    GPT-4o-mini GPT-3.5-turbo 비용이 60% 저렴
    Langchain V0.3x 부터 openAI가 별도 패키지로 분리되어 있어서 필요한 패키치 설치해야함: langchain-openai 필요
    토큰사용량 추적, 캐싱을 위한 langchain-community도 별도 설치
    환경변수 변수 관리 패키지 : python-dotenv
```

In [14]:
%pip install langchain-openai langchain-community python-dotenv openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [langchain-community]ngchain-community]
Note: you may need to restart the kernel to use updated packages.


In [15]:
# 환경변수 로드
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.7) #dotenv를 하면 자동으로 OPENAI_API_KEY 참조
prompt = "Langchain에 대해 한 문장으로 설명해줘"
result = llm.invoke(prompt)
result.content

'Langchain은 다양한 언어 모델과 데이터 소스를 통합하여 자연어 처리 애플리케이션을 구축할 수 있도록 돕는 프레임워크입니다.'

In [17]:
#사용량
result.usage_metadata

{'input_tokens': 18,
 'output_tokens': 36,
 'total_tokens': 54,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [ ]:
#콜백함수를 통한 누적 토큰 추적(get_openai_callback)
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫번째 호출
    res1 = llm.invoke('서울의 오늘 날씨는 어떤지 알려줘')
    print('응답1', res1.content[:10],'...')
    # 두번째 호출
    res2 = llm.invoke('파이썬으로 랭체인 사용법 알려줘')
    print('응답2', res2.content[:10],'...')

# 누적 토큰 사용량 출력 - 콜백 cb에는 블록 내 전체 토큰 사용량이 누적
# 총 토큰 수
print('총 토큰수:', cb.total_tokens)
# 프롬프트 토큰 수
print('프롬프트 토큰수:', cb.prompt_tokens)
# 응답 토큰 수
print('응답 토큰수:', cb.completion_tokens)
# 비용 계산
print('비용(USD):', cb.total_cost)

응답1 죄송하지만, 실시간 ...
응답2 랭체인(LangCh ...
총 토큰수: 661
프롬프트 토큰수: 37
응답 토큰수: 624
비용(USD): 0.00037995


In [20]:
# Langchain의 LLM 응답캐싱 (InMemory Cache, SQLiteCache)
# 사용하는 이유: 동일한 질문은 저장해뒀다가 응답에 사용
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCash 설정
set_llm_cache(InMemoryCache())

In [24]:
# 캐시 사용 전후를 비교, 같은 질문을 두 번 호출
query = "재미있는 유머 하나 알려줘"
# 첫 번째 호출(캐시에 없으면 api 호출 발생)
result1 = llm.invoke(query)
print(f'응답1 : {result1.content}')
print('*'*50)
# 두 번째 호출(동일한 query, 캐시를 확인하고 동일 질문이면 api 미호출)
result2 = llm.invoke(query)
print(f'응답2 : {result2.content}')


응답1 : 물고기가 수영을 잘하는 이유는 무엇일까요?

바로 "물 속에서 훈련"을 했기 때문이죠! 😂
**************************************************
응답2 : 물고기가 수영을 잘하는 이유는 무엇일까요?

바로 "물 속에서 훈련"을 했기 때문이죠! 😂


In [ ]:
# 실행시간 측정
import time
# 첫 번재 호출 시간
query = '점심메뉴 추천해줘'
start = time.time(); llm.invoke(query); end = time.time()
print(f'첫번째 호출시간: {end-start}')

start = time.time(); llm.invoke(query); end = time.time()
print(f'두번째 호출시간: {end-start}') #거의 0초 예상, invoke했지만 api안가고 그냥 cache 메모리 안에 있는걸 꺼낸거니까
# 이게 InMemory 방식. 시간비용 절약할 수 있고, 중복대답을 방지할 수 있다

첫번째 호출시간: 4.55459189414978
두번째 호출시간: 0.0015299320220947266


In [ ]:
# SQLite 캐시(디스크기반 캐시)
import os, time
from langchain_community.cache import SQLiteCache
# 기존 캐시 DB 삭제(.langchain.db 초기화)
# 첫번째 실행 후 주석처리
# if os.path.exists('.langchain.db'):
#     os.remove('.langchain.db')

# SQLite 캐시 설정(지정한 경로의 DB 파일을 생성 / 사용)
set_llm_cache(SQLiteCache(database_path='.langchain.db')) #클래스를 이용해 객체 생성 , langchain.db있으면 사용, 없으면 생성

# 동일한 query를 두번 호출해서 결과와 시간을 비교
query = "썰렁한 유머 하나 알려줘"
# 첫 번째 호출(캐시에 없으면 api 호출 발생)
start = time.time(); result1 = llm.invoke(query); end = time.time()
print(f'첫번째 호출시간: {end-start}')
print(f'응답1 : {result1.content}')

# 두 번째 호출(동일한 query, 캐시를 확인하고 동일 질문이면 api 미호출)
start = time.time(); result2 = llm.invoke(query); end = time.time()
print(f'두번째 호출시간: {end-start}')
print(f'응답2 : {result2.content}')

첫번째 호출시간: 1.5461149215698242
응답1 : 물고기가 학교에 가면 뭘 배우죠? 

"물리학"이요! 

(왜냐하면 물속에 있으니까요!)
두번째 호출시간: 0.001699209213256836
응답2 : 물고기가 학교에 가면 뭘 배우죠? 

"물리학"이요! 

(왜냐하면 물속에 있으니까요!)
